In [ ]:
import pandas as pd
import json
import re
from datetime import datetime

# Cargar el CSV con los datos de despliegue
df_deployment = pd.read_csv("final2_modified_sorted.csv")

# Inicializar estructuras
deployment_objects = []
node_uid_map = {}

# Función para limpiar cadenas para UIDs
def clean_str(s): 
    return re.sub(r'\W+', '_', str(s))

# Procesar filas del CSV
for _, row in df_deployment.iterrows():
    pod_name = str(row["pod"])
    node_name = str(row["instance"])
    microservice = str(row["Microservice"])
    abnormality = str(row["Abnormality class"])
    timestamp = str(row["timestamp"])
    experiment = int(row["Experiment"])
    cpu = float(row["container_cpu_usage_seconds_total"])
    memory = float(row["container_memory_working_set_bytes"])
    net_rx = float(row["container_network_receive_bytes_total"])
    net_tx = float(row["container_network_transmit_packets_total"])

    # Convertir timestamp a formato ISO
    try:
        timestamp_iso = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S").isoformat()
    except ValueError:
        timestamp_iso = timestamp

    node_uid = f"_:node_{clean_str(node_name)}"
    pod_uid = f"_:pod_{clean_str(pod_name)}_{clean_str(timestamp)}"

    if node_uid not in node_uid_map:
        node_obj = {
            "uid": node_uid,
            "dgraph.type": "Node",
            "node": {
                "name": node_name
            }
        }
        deployment_objects.append(node_obj)
        node_uid_map[node_uid] = True

    pod_obj = {
        "uid": pod_uid,
        "dgraph.type": "Pod",
        "pod": {
            "name": pod_name,
            "timestamp": timestamp_iso,
            "microservice": microservice,
            "abnormality": abnormality,
            "experiment": experiment,
            "cpu": cpu,
            "memory": memory,
            "net_rx": net_rx,
            "net_tx": net_tx
        },
        "isDeployedOn": {
            "uid": node_uid
        }
    }
    deployment_objects.append(pod_obj)

# Guardar en archivo JSON
with open("deployment_graph_cleaned.json", "w", encoding="utf-8") as f:
    json.dump(deployment_objects, f, indent=2)
